In [4]:
# ATSIT 2019: dimensional speech emotion recognition from text feature

# uncomment these to run on CPU only
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]=""

import numpy as np
import matplotlib.pyplot as plt
import pickle
import keras.backend as K
import pandas as pd
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, LSTM, TimeDistributed, Bidirectional, Embedding,GRU, Dropout, Flatten, concatenate, Convolution1D, BatchNormalization
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from keras.layers.convolutional import Conv2D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping
import codecs
from keras.optimizers import SGD, Adam, RMSprop
import io
import random as rn
import tensorflow as tf
from keras.layers.core import Dense, Activation
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)
# load feature and labels
feat = np.load('../../dimensionalSM/data/feat_34_hfs.npy')
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')
# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)

# standardization
scaled_feature = False
if scaled_feature == True:
    scaler = StandardScaler()
    scaler = scaler.fit(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaler.transform(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaled_feat.reshape(feat.shape[0], feat.shape[1], feat.shape[2])
    feat = scaled_feat
else:
    feat = feat

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1, keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss
import pickle
with open('../../dimensionalSM/IEMOCAP_full_release/data_collected_333.pickle', 'rb') as handle:
    data2 = pickle.load(handle)
    
x_train_mocap = []
counter = 0
for ses_mod in data2:
    x_head = ses_mod['mocap_head']
    if(x_head.shape != (200,18)):
        x_head = np.zeros((200,18))   
    x_head[np.isnan(x_head)]=0
    x_hand = ses_mod['mocap_hand']
    if(x_hand.shape != (200,6)):
        x_hand = np.zeros((200,6))   
    x_hand[np.isnan(x_hand)]=0
    x_rot = ses_mod['mocap_rot']
    if(x_rot.shape != (200,165)):
        x_rot = np.zeros((200,165))   
    x_rot[np.isnan(x_rot)]=0
    x_mocap = np.concatenate((x_head, x_hand), axis=1)
    x_mocap = np.concatenate((x_mocap, x_rot), axis=1)
    x_train_mocap.append( x_mocap )
    
x_train_mocap = np.array(x_train_mocap)
#x_train_mocap = x_train_mocap.reshape(-1,200,189,1)
x_train_mocap.shape
# remove outlier
# outlier = np.array([1674, 3427, 5086, 5093, 5096, 5104, 7821])
# mask = np.ones(len(vad), np.bool)
# mask[outlier] = 0
# vad = vad[mask]
# x_train_mocap = x_train_mocap[mask]


(10039, 200, 189)

In [6]:
# API model, if use RNN, first two rnn layer must return_sequences=True
def api_model(a,b,c):
    
    input_mocap = Input(shape=(200,189))
    model_mocap0 = BatchNormalization()(input_mocap)
    model_mocap1 = Convolution1D(64, 3, border_mode='same')(model_mocap0)
    model_mocap2 = Dropout(0.2)(model_mocap1)
    model_mocap3 = Activation('relu')(model_mocap2)
    model_mocap4 = Convolution1D(128, 3, border_mode='same')(model_mocap3)
    model_mocap5 = Dropout(0.2)(model_mocap4)
    model_mocap6 = Activation('relu')(model_mocap5)
    model_mocap7 = Convolution1D(256, 3, border_mode='same')(model_mocap6)
    model_mocap8 = Dropout(0.2)(model_mocap7) 
    model_mocap9 = Activation('relu')(model_mocap8)
    model_mocap10 = Convolution1D(128, 3, border_mode='same')(model_mocap9)
    model_mocap11 = Dropout(0.2)(model_mocap10)  
    model_mocap12 = Activation('relu')(model_mocap11)
    model_mocap13 = Convolution1D(64, 3, border_mode='same')(model_mocap12)
    model_mocap14 = Dropout(0.2)(model_mocap13) 
    model_mocap15 = Activation('relu')(model_mocap14)
    model_mocap = Flatten()(model_mocap15) 
    #model_mocap = Dropout(dropout_rate)(model_mocap16)

    target_names = ('v', 'a', 'd')
    model_combined = [Dense(1, name=name)(model_mocap) for name in target_names]

    model = Model(input_mocap, model_combined) 

    model.compile(loss=ccc_loss, 
                  loss_weights={'v': a, 'a': b, 'd': c},
                  optimizer='rmsprop', metrics=[ccc])
    return model

#def main(alpha, beta, gamma):
#model = api_model(1, 1, 1)
#odel = api_model(0.1, 0.5, 0.4)
#model.summary()
# main function for for LOSO


In [7]:
def main():
    model = api_model(1,1,1)
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model.fit( x_train_mocap[:7869], 
                      vad[:7869].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model.evaluate( x_train_mocap[7869:], vad[7869:].T.tolist())
    print(metrik[-3:])
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.1, 0.5, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(20):
        best = main()
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('mocap_CNN1D_7869-batch.csv', header=['time', 'v','a','d', 'ave'])


/root/anaconda3/envs/ser36/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, padding="same")`
  
/root/anaconda3/envs/ser36/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(128, 3, padding="same")`
  if __name__ == '__main__':
/root/anaconda3/envs/ser36/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(256, 3, padding="same")`
  if sys.path[0] == '':
/root/anaconda3/envs/ser36/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(128, 3, padding="same")`
  from ipykernel import kernelapp as app
/root/anaconda3/envs/ser36/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, padding="same")`


Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 20s 3ms/step - loss: 2.0745 - v_loss: 0.5468 - a_loss: 0.7639 - d_loss: 0.7642 - v_ccc: 0.4534 - a_ccc: 0.2361 - d_ccc: 0.2360 - val_loss: 2.8243 - val_v_loss: 0.8899 - val_a_loss: 0.9632 - val_d_loss: 0.9681 - val_v_ccc: 0.1066 - val_a_ccc: 0.0375 - val_d_ccc: 0.0317
Epoch 2/50
6295/6295 [==============================] - 17s 3ms/step - loss: 1.7269 - v_loss: 0.4185 - a_loss: 0.6592 - d_loss: 0.6494 - v_ccc: 0.5815 - a_ccc: 0.3409 - d_ccc: 0.3507 - val_loss: 2.5477 - val_v_loss: 0.8529 - val_a_loss: 0.8238 - val_d_loss: 0.8671 - val_v_ccc: 0.1363 - val_a_ccc: 0.1817 - val_d_ccc: 0.1343
Epoch 3/50
6295/6295 [==============================] - 16s 3ms/step - loss: 1.5799 - v_loss: 0.3891 - a_loss: 0.5949 - d_loss: 0.5960 - v_ccc: 0.6109 - a_ccc: 0.4051 - d_ccc: 0.4041 - val_loss: 2.5292 - val_v_loss: 0.8563 - val_a_loss: 0.8119 - val_d_loss: 0.8637 - val_v_ccc: 0.1355 - val_a_ccc: 0.19

/root/anaconda3/envs/ser36/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, padding="same")`
  
/root/anaconda3/envs/ser36/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(128, 3, padding="same")`
  if __name__ == '__main__':
/root/anaconda3/envs/ser36/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(256, 3, padding="same")`
  if sys.path[0] == '':
/root/anaconda3/envs/ser36/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(128, 3, padding="same")`
  from ipykernel import kernelapp as app
/root/anaconda3/envs/ser36/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, padding="same")`


Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 20s 3ms/step - loss: 2.0798 - v_loss: 0.5457 - a_loss: 0.7702 - d_loss: 0.7640 - v_ccc: 0.4542 - a_ccc: 0.2299 - d_ccc: 0.2361 - val_loss: 2.6162 - val_v_loss: 0.8902 - val_a_loss: 0.8628 - val_d_loss: 0.8606 - val_v_ccc: 0.1006 - val_a_ccc: 0.1429 - val_d_ccc: 0.1402
Epoch 2/50
6295/6295 [==============================] - 17s 3ms/step - loss: 1.6921 - v_loss: 0.4146 - a_loss: 0.6426 - d_loss: 0.6348 - v_ccc: 0.5851 - a_ccc: 0.3573 - d_ccc: 0.3654 - val_loss: 2.5316 - val_v_loss: 0.8781 - val_a_loss: 0.7959 - val_d_loss: 0.8665 - val_v_ccc: 0.1182 - val_a_ccc: 0.2143 - val_d_ccc: 0.1358
Epoch 3/50
6295/6295 [==============================] - 17s 3ms/step - loss: 1.5514 - v_loss: 0.3942 - a_loss: 0.5755 - d_loss: 0.5824 - v_ccc: 0.6057 - a_ccc: 0.4248 - d_ccc: 0.4180 - val_loss: 2.4900 - val_v_loss: 0.8698 - val_a_loss: 0.8039 - val_d_loss: 0.8221 - val_v_ccc: 0.1281 - val_a_ccc: 0.20

/root/anaconda3/envs/ser36/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, padding="same")`
  
/root/anaconda3/envs/ser36/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(128, 3, padding="same")`
  if __name__ == '__main__':
/root/anaconda3/envs/ser36/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(256, 3, padding="same")`
  if sys.path[0] == '':
/root/anaconda3/envs/ser36/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(128, 3, padding="same")`
  from ipykernel import kernelapp as app
/root/anaconda3/envs/ser36/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, padding="same")`


...

Train on 6295 samples, validate on 1574 samples
Epoch 1/50
1120/6295 [====>.........................] - ETA: 26s - loss: 2.7116 - v_loss: 0.8297 - a_loss: 0.9297 - d_loss: 0.9523 - v_ccc: 0.1703 - a_ccc: 0.0703 - d_ccc: 0.0477

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6295/6295 [==============================] - 17s 3ms/step - loss: 1.1855 - v_loss: 0.3087 - a_loss: 0.4468 - d_loss: 0.4299 - v_ccc: 0.6912 - a_ccc: 0.5531 - d_ccc: 0.5703 - val_loss: 2.3821 - val_v_loss: 0.8286 - val_a_loss: 0.7749 - val_d_loss: 0.7856 - val_v_ccc: 0.1694 - val_a_ccc: 0.2294 - val_d_ccc: 0.2191
Epoch 20/50
6295/6295 [==============================] - 17s 3ms/step - loss: 1.1820 - v_loss: 0.3048 - a_loss: 0.4459 - d_loss: 0.4311 - v_ccc: 0.6950 - a_ccc: 0.5543 - d_ccc: 0.5687 - val_loss: 2.3830 - val_v_loss: 0.8289 - val_a_loss: 0.7704 - val_d_loss: 0.7901 - val_v_ccc: 0.1672 - val_a_ccc: 0.2353 - val_d_ccc: 0.2145
Epoch 21/50
6295/6295 [==============================] - 17s 3ms/step - loss: 1.1798 - v_loss: 0.3038 - a_loss: 0.4462 - d_loss: 0.4296 - v_ccc: 0.6963 - a_ccc: 0.5538 - d_ccc: 0.5702 - val_loss: 2.3834 - val_v_loss: 0.8504 - val_a_loss: 0.7525 - val_d_loss: 0.7879 - val_v_ccc: 0.1447 - val_a_ccc: 0.2546 - val_d_ccc: 0.2173
Epoch 22/50
6295/6295 [===========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6295/6295 [==============================] - 17s 3ms/step - loss: 1.3494 - v_loss: 0.3366 - a_loss: 0.5076 - d_loss: 0.5048 - v_ccc: 0.6633 - a_ccc: 0.4923 - d_ccc: 0.4951 - val_loss: 2.4046 - val_v_loss: 0.8415 - val_a_loss: 0.7624 - val_d_loss: 0.8043 - val_v_ccc: 0.1520 - val_a_ccc: 0.2443 - val_d_ccc: 0.1991
Epoch 9/50
6295/6295 [==============================] - 17s 3ms/step - loss: 1.3161 - v_loss: 0.3263 - a_loss: 0.4926 - d_loss: 0.4969 - v_ccc: 0.6735 - a_ccc: 0.5073 - d_ccc: 0.5031 - val_loss: 2.4222 - val_v_loss: 0.8349 - val_a_loss: 0.7911 - val_d_loss: 0.8064 - val_v_ccc: 0.1662 - val_a_ccc: 0.2141 - val_d_ccc: 0.1975
Epoch 10/50
6295/6295 [==============================] - 17s 3ms/step - loss: 1.3243 - v_loss: 0.3321 - a_loss: 0.5048 - d_loss: 0.4873 - v_ccc: 0.6679 - a_ccc: 0.4951 - d_ccc: 0.5127 - val_loss: 2.4366 - val_v_loss: 0.8724 - val_a_loss: 0.7615 - val_d_loss: 0.8033 - val_v_ccc: 0.1181 - val_a_ccc: 0.2443 - val_d_ccc: 0.2009
Epoch 11/50
6295/6295 [============